<a href="https://colab.research.google.com/github/DilkiIshara/Hate-Speech-Detection-System/blob/main/Hate_Speech_Detection_CLL_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install tensorflow

In [4]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00


In [5]:
import csv
import re
from collections import defaultdict as ddict

import nltk
import numpy as np
import pandas as pd
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
# from transformers import (AdamW, XLMRobertaForSequenceClassification,
#                           XLMRobertaTokenizer, get_linear_schedule_with_warmup)
from transformers import XLMRobertaTokenizer, XLMRobertaModel, MarianMTModel, MarianTokenizer, AutoTokenizer,AutoModelForSequenceClassification, pipeline, AutoModelForSeq2SeqLM
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
    # Ensure consistent encoding
    # encoding_type = 'utf-8'

    # englishDataPath = '/content/drive/MyDrive/HSDData/EnglishDataSet.csv'
    # output_filename = '/content/drive/MyDrive/HSDData/train.tsv'

    # # Read the CSV file with explicit encoding
    # english_train_data = pd.read_csv(englishDataPath, encoding=encoding_type)

    # # Optionally inspect the first few rows of the data
    # print("Data from CSV:")
    # print(english_train_data.head())

    # # Save to TSV with the same encoding
    # train = english_train_data
    # train.to_csv(output_filename, index=False, sep='\t', header=True, encoding=encoding_type)

    # print(f"Data has been saved to {output_filename} successfully.")

    # # Verify the contents of the saved TSV
    # saved_data = pd.read_csv(output_filename, sep='\t', encoding=encoding_type)
    # print("Data from TSV:")
    # print(saved_data.head())



Load The Data set

In [7]:
# Ensure consistent encoding
encoding_type = 'utf-8'

englishDataPath = '/content/drive/MyDrive/HSDData/EnglishDataSet.csv'


try:
    english_train_data = pd.read_csv(englishDataPath, encoding=encoding_type)
except pd.errors.EmptyDataError:
    print("The file is empty or could not be loaded.")
except pd.errors.ParserError as e:
    print(f"Parser Error: {e}")

Display all the row data

In [8]:
english_train_data

,text,label-word,label
0,It should be legal for a husband to rape his wife,hate,1
1,dalits are lowlife scum,hate,1
2,dalits are lowlives,hate,1
3,It was a better world when women dare not ques...,hate,1
4,I don't work this hard so that those immigrant...,hate,1
...,...,...,...
40618,we should forcibly remove all the foreigners f...,hate,1
40619,we should forcibly remove all the immigrants f...,hate,1
40620,dalits are unworthy of the Gods love,hate,1
40621,dalits are worthless,hate,1


Clean Up the data Set and Display data

In [9]:
def cleaning_text(df, text_field):
  df[text_field] = df[text_field].str.lower().str.replace(r'[^\w\s]', '', regex=True)
  return df

train = cleaning_text(english_train_data, "text")
train

,text,label-word,label
0,it should be legal for a husband to rape his wife,hate,1
1,dalits are lowlife scum,hate,1
2,dalits are lowlives,hate,1
3,it was a better world when women dare not ques...,hate,1
4,i dont work this hard so that those immigrants...,hate,1
...,...,...,...
40618,we should forcibly remove all the foreigners f...,hate,1
40619,we should forcibly remove all the immigrants f...,hate,1
40620,dalits are unworthy of the gods love,hate,1
40621,dalits are worthless,hate,1


Save the clean Up data in train.tsv

In [10]:
output_filename = '/content/drive/MyDrive/HSDData/train.tsv'
train.to_csv(output_filename, index=False, sep='\t', header=True, encoding=encoding_type)

Split the Data into training and testing sets and save in separate files

In [11]:
trainDataSetFileName = '/content/drive/MyDrive/HSDData/train_DataSet.tsv'
testDataSetFileName = '/content/drive/MyDrive/HSDData/test_DataSet.tsv'

train_DataSet = train.sample(frac=0.8, random_state=73)
test_DataSet = train.drop(train_DataSet.index)


train_DataSet.to_csv(trainDataSetFileName, index=False, sep='\t', header=True, encoding=encoding_type)
test_DataSet.to_csv(testDataSetFileName, index=False, sep='\t', header=True, encoding=encoding_type)

In [12]:
label_categories = {
    'nothate':0,
    'hate':1
}

def load_dataset():
  lines = []
  with open(trainDataSetFileName, "r", encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for line in reader:
      lines.append(line)
      # print("Data" + line[0])
    texts=[]
    label =[]
    dic_data = ddict(list)
    for (i,line) in enumerate(lines):
      if i==0:
        continue

      text = line[0]
      labelValue = line[1]

      # print("Data" + text)
      # print("Label" + labelValue)

      texts.append(text)
      label.append(label_categories[labelValue])
    label_list = ["hate", "nothate"]
    dic_data['texts']=texts
    dic_data['label']=label
    print(len(texts))
    print(len(label))
    return dic_data,label_list

train_DataList, label_List= load_dataset()

32498
32498


In [13]:
data = pd.DataFrame(
    {
        'EnglishData' : train_DataList['texts'],
        'Label' : train_DataList['label']
    }
)

data


,EnglishData,Label
0,0ld p3opl3 are trsh,1
1,coffee is fucking disgusting,0
2,faggots are great at being beastly,1
3,i dont want to sound racist or anything but as...,1
4,fuck seaworld orcas dont belong in captivity,0
...,...,...
32493,britain has has become much more diverse and e...,0
32494,i feel like as a dyke i should be offended but...,0
32495,so pleased my son isnt a spazz,1
32496,hindus hate all other religions,1


In [14]:
# Load tokenizer and model for XLM-RoBERTa
xlm_roberta_model_name = "xlm-roberta-base"
xlm_roberta_tokenizer = XLMRobertaTokenizer.from_pretrained(xlm_roberta_model_name)
xlm_roberta_model = XLMRobertaModel.from_pretrained(xlm_roberta_model_name)

# Load tokenizer and model for translation (English to Sinhala)
translation_model_name = "thilina/mt5-sinhalese-english"
translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [15]:
data['EnglishData']

,EnglishData
0,0ld p3opl3 are trsh
1,coffee is fucking disgusting
2,faggots are great at being beastly
3,i dont want to sound racist or anything but as...
4,fuck seaworld orcas dont belong in captivity
...,...
32493,britain has has become much more diverse and e...
32494,i feel like as a dyke i should be offended but...
32495,so pleased my son isnt a spazz
32496,hindus hate all other religions


 Compute embeddings for English text

In [16]:
data['EnglishData']

# Function to compute embeddings
def get_embedding(text, tokenizer, model):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    last_hidden_state = output.last_hidden_state
    embedding = torch.mean(last_hidden_state, dim=1).squeeze()
    return embedding.numpy()

# Compute embeddings for English text
data['english_embedding'] = data['EnglishData'].apply(lambda x: get_embedding(x, xlm_roberta_tokenizer, xlm_roberta_model))

data['english_embedding']

english_embedding = '/content/drive/MyDrive/HSDData/english_embedding.tsv'
data.to_csv(output_fenglish_embeddingilename, index=False, sep='\t', header=True, encoding=encoding_type)

,english_embedding
0,"[0.015025277, 0.002972979, 0.020253409, -0.013..."
1,"[-0.025379706, 0.062638775, 0.009610087, 0.037..."
2,"[0.0036100645, 0.054091666, -0.01471246, 0.067..."
3,"[-0.020647164, 0.021076692, 0.002199945, 0.024..."
4,"[-0.04265948, 0.068035014, -0.0028829032, -0.0..."
...,...
32493,"[-0.02697971, 0.015191623, 0.0074848738, 0.014..."
32494,"[-0.03764168, 0.03053095, -0.0019753522, 0.016..."
32495,"[-0.037249945, 0.021588963, 0.016294857, 0.032..."
32496,"[-0.04517668, -0.0064641386, 0.013167163, 0.02..."


In [17]:
#Translate English texts to Sinhala

In [18]:
# Function to translate text
def translate_to_sinhala(texts, tokenizer, model):
  # Prepare the input text
  text_to_translate = "Hello, how are you?"
  # Tokenize the input text for translation
  inputs = tokenizer.encode(text_to_translate, return_tensors="pt")
  # Generate translation
  translated_tokens = model.generate(inputs, max_length=100)
  # Decode the translated tokens to get the output
  translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
  print(translated_text)
    # translated_texts = []
    # for text in texts:
        # translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
        # translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
        # translated_texts.append(translated_text)


        #pipe = pipeline("translation", model=translation_model_name)


    # return translated_texts
#data['translated_sinhala'] =
translate_to_sinhala(data['EnglishData'], translation_tokenizer, translation_model)




හෙලෝ, කොහොමද?


In [ ]:
# Function to translate text
translator = pipeline("translation", model=translation_model_name)
def translate_to_sinhala(texts, tokenizer, model):
    translated_texts = []
    for text in texts:
      translated_text = translator(text)
      translated_texts.append(translated_text)



    # return translated_texts
data['translated_sinhala'] = translate_to_sinhala(data['EnglishData'], translation_tokenizer, translation_model)
data['translated_sinhala']

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agno

/////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# # Initialize the translator pipeline once
# translatorNew = pipeline("translation", model=translation_model_name)

# def translate_to_sinhala(texts, translator):
#     # Batch translation for all texts at once
#     translated_outputs = translator(texts, src_lang="en", tgt_lang="si")

#     # Extract the translated text from the output dictionaries
#     translated_texts = [output['translation_text'] for output in translated_outputs]

#     return translated_texts

# # Apply translation and store the result in the DataFrame
# data['translated_sinhalaText'] = translate_to_sinhala(data['EnglishData'], translatorNew)

# # Display the translated data
# data['translated_sinhalaText']

In [ ]:
# # Function to translate text
# def translate_to_sinhala(texts, tokenizer, model):
#     translator = pipeline("translation", model=translation_model_name)
#     translated_text = translator(text, src_lang="en", tgt_lang="si")
#     print(translated_text)



# translate_to_sinhala(data['EnglishData'], translation_tokenizer, translation_model)


In [ ]:
# # Function to translate text
# def translate_to_sinhala(texts, tokenizer, model):
#     text_to_translate = "Hello, how are you?"
#     translator = pipeline("translation", model=translation_model)
#     translated_text = translator(text_to_translate)
#     print(translated_text)



# translate_to_sinhala(data['EnglishData'], translation_tokenizer, translation_model)

In [ ]:
# # Function to translate text
# def translate_to_sinhala(texts, tokenizer, model):
#     text_to_translate = "Hello, how are you?"
#     translator = pipeline("translation", model=translation_model)
#     translated_text = translator(text_to_translate)
#     print(translated_text)



# translate_to_sinhala(data['EnglishData'], translation_tokenizer, translation_model)

In [ ]:
# # Load tokenizer and model for translation (English to Sinhala)
# translation_model_name = "thilina/mt5-sinhalese-english"
# translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
# translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)
# translator = pipeline("translation", model="thilina/mt5-sinhalese-english")

# # Function to translate text
# def translate_to_sinhala(tokenizer, model):
#     # Load the translation pipeline for the mt5-sinhalese-english model

#     # Provide a Sinhalese sentence to translate to English
#     result = translator("ඔබ කෙසේද?")
#     print(result)
#     result = translator("I am Dilki")
#     print(result)



# translate_to_sinhala(translation_tokenizer, translation_model)

In [ ]:
# import torch
# print(torch.cuda.is_available())